In [1]:
import pandas as pd

In [4]:
df_fiis = pd.read_parquet('df_fiis.parquet')

In [5]:
df_fiis

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,DY Patrimonial,Variação Patrimonial,Rentab. Patr. Período,Rentab. Patr. Acumulada,Quant. Ativos,Volatilidade,Num. Cotistas,Tax. Gestão,Tax. Performance,Tax. Administração
0,AAGR11,Indefinido,12299,"13.345,09",NaN,146,"1,59 %","1,59 %","6,32 %","16,17 %",...,"0,00 %","0,00 %","0,00 %","0,00 %",0,6127,0,None,None,None
1,AAZQ11,Indefinido,849,"413.207,36",97.0,012,"1,47 %","4,51 %","9,22 %","18,95 %",...,"1,39 %","0,00 %","0,00 %","0,00 %",0,1646,30.064,None,"1,20 % a.a","10,00 % a.a"
2,ABCP11,Shoppings,7808,"76.833,91",72.0,065,"0,86 %","2,39 %","4,93 %","9,71 %",...,"0,59 %","20,08 %","20,79 %","29,39 %",1,2619,14.130,None,"0,10 % a.a",None
3,AFHI11,Papéis,9456,"798.950,27",100.0,101,"1,06 %","3,21 %","6,72 %","13,71 %",...,"1,07 %","-0,45 %","0,61 %","6,92 %",14,958,37.566,None,"1,00 % a.a",None
4,AGRX11,Indefinido,802,"264.792,77",75.0,012,"1,49 %","4,49 %","9,22 %","18,06 %",...,"1,13 %","0,00 %","0,00 %","0,00 %",0,2328,18.900,None,"1,00 % a.a","10,00 % a.a"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,YUFI11B,Indefinido,None,None,NaN,063,"0,00 %","0,00 %","0,00 %","0,00 %",...,"0,60 %","-0,01 %","0,59 %","6,32 %",0,None,0,None,None,None
544,ZAGH11,Serviços Financeiros Diversos,1011,"25.425,55",108.0,006,"0,62 %","1,83 %","3,62 %","7,48 %",...,None,None,None,None,1,6539,461,None,"1,50 % a.a","20,00 % a.a"
545,ZAVC11,Indefinido,924,"14.005,91",89.0,013,"0,00 %","2,69 %","7,15 %","16,50 %",...,None,None,None,None,1,6995,533,"1,25 % a.a",None,"20,00 % a.a"
546,ZAVI11,Indefinido,992,"60.615,45",7.0,110,"1,16 %","3,66 %","7,67 %","15,79 %",...,None,None,None,None,9,9530,2.338,None,"1,25 % a.a","20,00 % a.a"
